# 分布式训练

<img src="./pic/分布式训练.PNG" width=300 height=300>


## GPU机器架构

- 少在机器之间做通讯
<img src="./pic/GPU机器架构.PNG" width=300 height=300>


## 计算一个小批量

1. 每个计算服务器读取小批量中的一部分样本（总共100个样本，每个服务器分50个样本）
2. 进一步将数据切分到每个GPU上（单个服务器50个样本，分给每个GPU 25个样本）
3. 每个worker从参数服务器那里获取模型参数
4. 复制参数到每个GPU上
5. 每个GPU计算梯度
6. 将所有GPU上的梯度求和（在本地做all-reduce）
7. 梯度传回服务器（每个服务器只用发一遍梯度和）
8. 每个服务器对梯度求和，并更新参数
9. 进行下一批量的计算

<table>
    <tr>
    <td><img src="./pic/step1.PNG" width=300 height=300></td>
    <td><img src="./pic/step2.PNG" width=300 height=300></td>
    <td><img src="./pic/step3.PNG" width=300 height=300></td>
    <td><img src="./pic/step4.PNG" width=300 height=300></td>
    </tr>
    <tr>
    <td><img src="./pic/step5.PNG" width=300 height=300></td>
    <td><img src="./pic/step6.PNG" width=300 height=300></td>
    <td><img src="./pic/step7.PNG" width=300 height=300></td>
    <td><img src="./pic/step8.PNG" width=300 height=300></td>
    </tr>
</table>

## 同步SGD

- 这里每个worker都是同步计算一个批量，称为同步SGD
- 假设有n个GPU，每个GPU每次处理b个样本，那么同步SGD等价于在单GPU运行批量大小为nb的SGD
- 在理想情况下，n个GPU可以得到相对单个GPU的n倍加速


## 性能
- 增大批量使得t1增大，t2不变（t1比t2大20%-30%）
- 更多计算：更多epoch
<img src="./pic/性能.PNG" width=300 height=300>

## 性能的权衡
- 系统性能随批量增多而增多，但会达到饱和
- 训练有效性随批量增多而减少，（在lr不变的情况下）收敛变慢（**因为批量大，迭代次数少，所以loss有波动**），需要更多的epoch来达到需要的精度
    - 增大批量b为啥会需要更多的epoch来训练？
    - 当mini-batch 是整个数据集的时候，就退化成了 Gradient Descent，这样的话，反而收敛速度慢。
    - batch 增大导致的计算 batch 代价变大。如果盲目增大mini-batch size 确实是迭代次数减少了，但是计算时间反而会增加，因为每次计算代价大了。

<img src="./pic/性能的权衡.PNG" width=300 height=300>


## 实践时的建议
- GPU-GPU的带宽：好的主板
- 机器-机器的带宽：好的服务器
- FLOP / model size = 越大越好
    - model size几乎等价于每做一次FLOP需要读多少个bit（数据）
    - Inception有大量的3x3卷积，模型更小，更有利于做并行
<img src="./pic/实践时的建议.PNG" width=400 height=400>
- 当批量很大时，需要改进优化算法
    - 比如增大lr
    - 调整优化算法使得更适用于超大的批量大小
    - 超大的批量大小怎样支持超大的lr

## 总结
- 分布式同步数据并行是多GPU数据并行在多机器上的拓展
    - 多GPU并行：在单机器内部运行
    - 多机器并行：在网络上运行
- 网络通讯通常是瓶颈
- 需要主要使用特别大的批量大小时，收敛效率
- 更复杂的分布式有异步、模型并行（做的不多）


### 假设N个分类，批量大小不要超过10xN
    - MNIST 10个类，批量大小 = 128
    - ImageNet 1000个类，批量大小 = 10w？